In [ ]:
import warnings
warnings.filterwarnings('ignore')
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
!pip install pycld2

In [ ]:
import json
import pickle
import pandas as pd
import re
import pycld2 as cld2
#with open('22gpt3.json') as f:
    # returns JSON object as
    # a dictionary
    #data = json.load(f)
data=pd.read_csv('/content/profs_for_graph.csv',index_col=False)
test_data=[]
other_data=[]
data=data.drop('Unnamed: 0',axis=1)
print(data.head())
# Iterating through the json
# list
data = data.reset_index()  # make sure indexes pair with number of rows
for index, row in data.iterrows():
    #c=data[k]['content']
    if pd.notnull(row['text']) is True:
      c=row['text']
      c=(c.encode('ascii','ignore')).decode()
      c= re.sub('[!,*)@#%(&$_?.^]', '', c)
      c= re.sub('\n','',c)
      c= re.sub('\t','',c)
      c= re.sub(r'http\S+', '', c)
      _,_,details=cld2.detect(c)
      if details[0][1]=='en' and details[0][2]>90:
          test_data.append(c)
            # other_data.append({'name':data[k]['user']['username'],
            # 'verified':data[k]['user']['verified'],
            # 'retweets':data[k]['retweetCount'],
            # 'likes':data[k]['likeCount']})

csv_data=pd.DataFrame(test_data)
csv_data.to_csv('profs.csv',index=False)

             id  timestamp_utc                 local_time user_screen_name  \
0  1.518949e+18            NaN  2022-04-26T13:43:37+00:00        beenwrekt   
1           NaN            NaN                        NaN              NaN   
2  1.518948e+18            NaN  2022-04-26T13:40:26+00:00        beenwrekt   
3           NaN            NaN                        NaN              NaN   
4  1.518942e+18            NaN  2022-04-26T13:16:52+00:00        beenwrekt   

                                                text  possibly_sensitive  \
0  @schmangee He traces out a repeated pattern in...                 0.0   
1                                                NaN                 NaN   
2  @schmangee Sadly, I find Tim Wu's arguments of...                 0.0   
3                                                NaN                 NaN   
4  @Bernstein Importantly, in the intervening yea...                 0.0   

   retweet_count  like_count  reply_count lang  ...  place_type  \
0      

In [ ]:
import random
text_data = []
with open('profs.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['neurips', '841io', 'lustmord', 'tribute', 'influence', 'interest', 'career']
['ifihadastick', 'wespegden', 'exactly', 'finding', 'implausible', 'suggest', 'upstream', 'error']
['anya1anya', 'available', 'online', 'means', 'preprint']
['angrycardio', 'winner', 'whoever', 'close', 'oclock']
['bergerbell', 'wespegden', 'devious', 'engineer', 'permanent', 'republicnan', 'control', 'explain', '1/348']
['ocramzyo', 'big', 'mistake', 'thinking', 'machine', 'learning', 'automate', 'things', 'people']
['boehninglab', 'reddit', 'downvotes']
['ajlamesa', 'agree', 'first', 'point', 'concede', 'end', 'mandatory', 'masking']
['zartmo3', 'schmangee', 'keegantweets', 'ajlamesa', 'clarify', 'argument', 'embracing', 'oster', 'group', 'epidemiologist', 'quote', 'later', 'article', 'tweet', 'disagree', 'twitter', 'article']
['thenickfoy', 'tweet', 'nail']
['plot', 'design', 'using', 'rectangular', 'block', 'single', 'section', 'foundation', 'paper', 'pieceable', 'plottertwitter', 'axidraw', 'quilting', 

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [ ]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

### Try 5 topics

In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.009*"great" + 0.005*"thank" + 0.005*"excite" + 0.005*"thanks"')
(1, '0.010*"devices" + 0.008*"author" + 0.006*"study" + 0.006*"chair"')
(2, '0.009*"research" + 0.007*"please" + 0.007*"tcuarch" + 0.007*"interest"')
(3, '0.010*"great" + 0.010*"research" + 0.007*"paper" + 0.007*"would"')
(4, '0.011*"paper" + 0.008*"seminar" + 0.006*"group" + 0.006*"review"')


In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(35, 1), (36, 1), (187, 1), (948, 1)]
[(0, 0.040160708), (1, 0.040175147), (2, 0.34663936), (3, 0.040600535), (4, 0.5324243)]


In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.004*"looking" + 0.004*"interest" + 0.004*"thank" + 0.004*"learning"')
(1, '0.007*"paper" + 0.006*"really" + 0.005*"please" + 0.005*"better"')
(2, '0.010*"research" + 0.009*"great" + 0.008*"group" + 0.005*"paper"')


In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.010*"group" + 0.010*"python" + 0.007*"every" + 0.007*"nicolacullum"')
(1, '0.015*"challenge" + 0.011*"great" + 0.008*"place" + 0.008*"devices"')
(2, '0.014*"tcuarch" + 0.011*"great" + 0.011*"first" + 0.011*"research"')
(3, '0.010*"paper" + 0.010*"definitely" + 0.010*"congrats" + 0.010*"award"')
(4, '0.011*"thanks" + 0.011*"student" + 0.011*"great" + 0.011*"vision"')
(5, '0.009*"application" + 0.009*"award" + 0.009*"instagram" + 0.006*"important"')
(6, '0.008*"control" + 0.008*"please" + 0.008*"protect" + 0.008*"learn"')
(7, '0.011*"really" + 0.011*"advisor" + 0.007*"would" + 0.007*"activity"')
(8, '0.019*"paper" + 0.008*"author" + 0.008*"stand" + 0.008*"least"')
(9, '0.012*"reviewer" + 0.009*"research" + 0.009*"paper" + 0.009*"later"')


### pyLDAvis

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [ ]:
!pip install pyLDAvis
!pip install pyLDAvis.gensim

ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display = gensimvis.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display = gensimvis.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)